In [8]:
import pandas as pd
import math

In [2]:
df = pd.read_csv('/home/gemmara/Downloads/supermarket.zip', compression='zip')
df

,Customer Id,Date,Product
0,JKML5ZWXP0XQ,2020-07-20,tropical fruit
1,YOGWQIUPDAJ7,2020-04-30,whole milk
2,Q3NL22281AF6,2020-09-18,pip fruit
3,C59GRTLMZQAU,2020-12-11,other vegetables
4,WUSF8LTEQZ5L,2020-01-02,whole milk
...,...,...,...
32705,PM1BQU9H9S2Q,2019-06-05,oil
32706,6AWIAP3ZV551,2019-08-10,sliced cheese
32707,LRWVFLSC8X5B,2019-02-23,candy
32708,8KYRMELYKNIH,2019-04-16,cake bar


تعداد محصولات یکتا در کل دادگان، چندتاست؟

In [14]:
df.Product.unique().size

137

میزان فروش به ازای هر روز به صورت میانگین چند است (تا دو رقم بعد از اعشار)؟


In [15]:
math.floor(df.groupby('Date').apply(lambda group: group.shape[0]).mean() * 100) / 100
# round(df.shape[0] / df['Date'].nunique())

44.93

چهار محصولی که کمتر از بقیه در سبد مشتریان قرار گرفته‌اند، کدامند (ترتیب مهم نیست)؟

In [16]:
df.groupby(['Product']).apply(lambda group: group.shape[0]).sort_values()[:4]

Product
kitchen utensil          1
preservation products    1
baby cosmetics           3
bags                     4
dtype: int64

پنج مشتری‌ای که در سال ۲۰۲۰، بیشترین تعداد "سبد" را داشته‌اند، کدامند (ترتیب مهم نیست)؟

In [6]:
df[df['Date'].str[:4] == '2020'].groupby(['Customer Id'])['Date'].nunique().sort_values(ascending=False)[:5]

Customer Id
M722MPITBY07    7
E7FD9BU4OK0F    7
0ZRF32AJ06BC    7
M2DT2Z00NQCH    7
QBUS9NZY39BE    7
Name: Date, dtype: int64

کدام روز هفته، بیشترین تعداد محصول فروش رفته‌است؟


In [7]:
df['Day_of_Week'] = pd.to_datetime(df['Date'], format='%Y-%m-%d').dt.day_name()
df['Day_of_Week'].max()

'Wednesday'